# W200 Project_2: Relationship Between Inequality and Globalization

Research question to answer:
- What is the relationship between the level of exposure of a country to globalization (world trade) and its ability to reduce income inequality?
- Hypothesis: More exposure to globalization leads to the reduction of income inequality

# Dataset preparation

In this jupyter notebook, I depart from Isabel's dataset and add to it the following variables:
* country_class: if a country is considered a developed or developing economy
* trade: amount of exports + imports of goods and services
* gdp: gdp amount in nominal, current terms
* trade_ratio: trade as a % of gdp

In [1]:
# importing numpy, pandas and matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# configuring some pandas options: float display and full display of dataset
# as our datasets are small I still find it worth to visualize the whole thing
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', None)

#setting matplotlib to display charts in this notebook
%matplotlib inline

# importing Isabel's dataset with gini per country
gini_df = pd.read_csv('final_gini_table.csv')

# Adding country_class

In [2]:
# importing Human Development Index by country
# from http://hdr.undp.org/en/indicators/137506
hdi_df = pd.read_csv('Human Development Index (HDI).csv')

# picking only the columns that are useful for us
hdi2_df = hdi_df[['Country', '2018']]

# renaming country column
hdi2_df.rename(columns = {'Country': 'country'}, inplace = True)

# stripping country names (they came with a space before first characters)
hdi2_df['country'] = hdi2_df['country'].str.strip()

# manually changing some country names to match with Isabel's dataset
hdi2_df.loc[21, 'country'] = 'Bolivia'
hdi2_df.loc[63, 'country'] = 'Gambia, The'
hdi2_df.loc[75, 'country'] = 'Hong Kong'
hdi2_df.loc[80, 'country'] = 'Iran'
hdi2_df.loc[91, 'country'] = 'Korea, Republic of'
hdi2_df.loc[114, 'country'] = 'Moldova'
hdi2_df.loc[142, 'country'] = 'Russia'
hdi2_df.loc[185, 'country'] = 'Venezuela'
hdi2_df.loc[186, 'country'] = 'Vietnam'

# replacing string characters for NaN in the 2018 Series
hdi2_df['2018'].replace('..', np.NaN, inplace = True)

# changing the dtype to 'float64'
hdi2_df['2018'] = hdi2_df['2018'].astype('float64')

# adding variable country_class: developed or developing country
# countries with HDI >= 0.85 are considered developed, all others are developing
hdi2_df['country_class'] = ['developed' if hdi >= 0.85 else 'developing' for hdi in hdi2_df['2018']]

# dropping hdi column
hdi3_df = hdi2_df.drop('2018', axis = 1)

# left-merging country_class with Isabel's dataset
gini2_df = gini_df.merge(hdi3_df, how = 'left', on = 'country')

# there are few countries present in Isabel's dataset that are not on my HDI dataset
not_matching = gini2_df[gini2_df['country_class'].isnull()]['country']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

# Adding trade_ratio

In [3]:
# importing trade as % of GDP dataset from World Bank
# from https://data.worldbank.org/indicator/NE.TRD.GNFS.ZS
trade_df = pd.read_excel('API_NE.TRD.GNFS.ZS_DS2_en_excel_v2_1217697.xls', sheet_name = 'Data', header = 3)

# removing columns that are not useful for us
trade_df.drop(trade_df.columns[2:53], axis = 1, inplace = True)
trade_df.drop(trade_df.columns[-1], axis = 1, inplace = True)

# renaming country and code columns
trade_df.rename(columns = {'Country Name':'country', 'Country Code': 'code3'}, inplace = True)

# converting years in a single column with datetime format
years = [str(x) for x in range(2009, 2019)] 
trade2_df = trade_df.melt(id_vars = ['country', 'code3'], value_vars = years, var_name = 'year', value_name = 'trade_ratio')
trade2_df['year'] = pd.to_datetime(trade2_df['year'], format = '%Y')

In [4]:
# renaming gini2_df year columns
dict_cols = {'g2009': '2009', 'g2010': '2010', 'g2011': '2011', 'g2012': '2012', 'g2013': '2013',
             'g2014': '2014', 'g2015': '2015', 'g2016': '2016', 'g2017': '2017', 'g2018': '2018'}
gini2_df.rename(columns = dict_cols, inplace = True)

# converting gini2_df years in a single column with datetime format
id_vars = ['country', 'code3', 'code2', 'region', 'sub_region', 'gini_trend', 'country_class']
years = [str(x) for x in range(2009, 2019)]
gini3_df = gini2_df.melt(id_vars = id_vars, value_vars = years, var_name = 'year', value_name = 'gini')
gini3_df['year'] = pd.to_datetime(gini3_df['year'], format = '%Y')

In [5]:
# left-merging trade_ratio with Isabel's dataset
gini4_df = gini3_df.merge(trade2_df, how = 'left', on = ['code3', 'year'])

# merge was perfect, except for Taiwan which was not present on trade_ratio dataset
not_matching2 = gini4_df[gini4_df['country_y'].isnull()]

# dropping duplicate country column, and renaming original one back to 'country'
gini5_df = gini4_df.drop('country_y', axis = 1)
gini5_df.rename(columns = {'country_x': 'country'}, inplace = True)

# Adding gdp

In [6]:
# importing GDP dataset from World Bank
# from https://data.worldbank.org/indicator/NY.GDP.MKTP.CD
gdp_df = pd.read_excel('API_NY.GDP.MKTP.CD_DS2_en_excel_v2_1282533.xls', sheet_name = 'Data', header = 3)

# removing columns that are not useful for us
gdp_df.drop(gdp_df.columns[2:53], axis = 1, inplace = True)
gdp_df.drop(gdp_df.columns[-1], axis = 1, inplace = True)

# renaming country and code columns
gdp_df.rename(columns = {'Country Name':'country', 'Country Code': 'code3'}, inplace = True)

# converting years in a single column with datetime format
years = [str(x) for x in range(2009, 2019)] 
gdp2_df = gdp_df.melt(id_vars = ['country', 'code3'], value_vars = years, var_name = 'year', value_name = 'gdp')
gdp2_df['year'] = pd.to_datetime(gdp2_df['year'], format = '%Y')

In [7]:
# left-merging gdp dataset with Isabel's dataset
gini6_df = gini5_df.merge(gdp2_df, how = 'left', on = ['code3', 'year'])

# merge was perfect, except for Taiwan which was not present on gdp dataset
not_matching3 = gini6_df[gini6_df['country_y'].isnull()]

# dropping duplicate country column, and renaming original one back to 'country'
gini7_df = gini6_df.drop('country_y', axis = 1)
gini7_df.rename(columns = {'country_x': 'country'}, inplace = True)

# Adding trade

In [8]:
# trade in nominal terms is the multiplication of gdp * trade_ratio
gini7_df['trade'] = gini7_df['trade_ratio'] * gini7_df['gdp'] / 100

# exporting .csv file
gini7_df.to_csv("final_gini_table_with_new_variables.csv")

In [9]:
gini7_df

,country,code3,code2,region,sub_region,gini_trend,country_class,year,gini,trade_ratio,gdp,trade
0,Afghanistan,AFG,AF,Asia,Southern Asia,-0.40,developing,2009-01-01,nan,49.66,"12,439,087,076.77","6,177,676,769.48"
1,Armenia,ARM,AM,Asia,Western Asia,0.10,developing,2009-01-01,31.76,57.27,"8,647,936,747.99","4,953,039,680.05"
2,Australia,AUS,AU,Oceania,Australia and New Zealand,0.24,developed,2009-01-01,nan,45.80,"927,805,183,330.88","424,915,329,112.06"
3,Austria,AUT,AT,Europe,Western Europe,-0.78,developed,2009-01-01,34.48,87.06,"400,172,297,860.52","348,398,930,258.41"
4,Bangladesh,BGD,BD,Asia,Southern Asia,0.23,developing,2009-01-01,nan,40.09,"102,477,791,472.39","41,086,212,108.88"
5,Barbados,BRB,BB,Americas,Caribbean,-1.86,developing,2009-01-01,nan,89.38,"4,465,500,000.00","3,991,469,500.00"
6,Belarus,BLR,BY,Europe,Eastern Europe,-0.35,developing,2009-01-01,27.69,108.64,"50,874,078,052.27","55,267,382,742.57"
7,Belgium,BEL,BE,Europe,Western Europe,-0.80,developed,2009-01-01,32.29,135.41,"481,345,929,424.84","651,771,464,295.64"
8,Benin,BEN,BJ,Africa,Western Africa,1.44,developing,2009-01-01,nan,44.70,"9,699,586,794.45","4,335,970,305.36"
9,Bhutan,BTN,BT,Asia,Southern Asia,-0.27,developing,2009-01-01,nan,109.57,"1,234,014,291.82","1,352,104,005.14"
